In [1]:
import pandas as pd

Источник данных: https://www.kaggle.com/datasets/gopinath15/friends-netflix-script-data

In [2]:
df = pd.read_csv('Friends.csv')
df

,Text,Speaker,Episode,Season,Show
0,Originally written by Marta Kauffman and David...,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
1,Transcribed by guineapig.,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
3,There's nothing to tell! He's just some guy I...,MONICA,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
4,"C'mon, you're going out with the guy! There's...",JOEY,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
...,...,...,...,...,...
69969,Then I'm happy too. (They're still hugging - ...,Ross,Episode-15-The One Where Estelle Dies,Season-10,Friends
69970,COMMERCIAL BREAK,NaN,Episode-15-The One Where Estelle Dies,Season-10,Friends
69971,Estelle's memorial service. Joey is giving a ...,[Scene,Episode-15-The One Where Estelle Dies,Season-10,Friends
69972,Thank you all for coming. We're here today to...,Joey,Episode-15-The One Where Estelle Dies,Season-10,Friends


Удаляем столбцы с названием эпизода, сезона и шоу

In [3]:
df = df.drop(['Show', 'Season', 'Episode'], axis=1)
df

,Text,Speaker
0,Originally written by Marta Kauffman and David...,NaN
1,Transcribed by guineapig.,NaN
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1
3,There's nothing to tell! He's just some guy I...,MONICA
4,"C'mon, you're going out with the guy! There's...",JOEY
...,...,...
69969,Then I'm happy too. (They're still hugging - ...,Ross
69970,COMMERCIAL BREAK,NaN
69971,Estelle's memorial service. Joey is giving a ...,[Scene
69972,Thank you all for coming. We're here today to...,Joey


В датасете есть пропущенные значения для говорящего персонажа. Сначала я решил их удалить, но после подумал, что они могут служить разделителями между отдельными диалогами, не связанными контекстом. Поэтому решил провести анализ, какие реплики предшествуют и следуют за пропущенными значениями

In [4]:
df['PrevText'] = df['Text'].shift(1)
df['NextText'] = df['Text'].shift(-1)
df

,Text,Speaker,PrevText,NextText
0,Originally written by Marta Kauffman and David...,NaN,None,Transcribed by guineapig.
1,Transcribed by guineapig.,NaN,Originally written by Marta Kauffman and David...,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1,Transcribed by guineapig.,There's nothing to tell! He's just some guy I...
3,There's nothing to tell! He's just some guy I...,MONICA,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,"C'mon, you're going out with the guy! There's..."
4,"C'mon, you're going out with the guy! There's...",JOEY,There's nothing to tell! He's just some guy I...,So does he have a hump? A hump and a hairpiece?
...,...,...,...,...
69969,Then I'm happy too. (They're still hugging - ...,Ross,"Oh! Oh, I'm so happy.",COMMERCIAL BREAK
69970,COMMERCIAL BREAK,NaN,Then I'm happy too. (They're still hugging - ...,Estelle's memorial service. Joey is giving a ...
69971,Estelle's memorial service. Joey is giving a ...,[Scene,COMMERCIAL BREAK,Thank you all for coming. We're here today to...
69972,Thank you all for coming. We're here today to...,Joey,Estelle's memorial service. Joey is giving a ...,THE END


Видно, что пропущенным значениям соответствуют рекламные паузы, метки начала и окончания эпизодов

In [5]:
df[df['Speaker'].isna()]['Text'].value_counts().head(10)

Text
Commercial Break    137
Opening Credits     127
End                 121
END                  73
Ending Credits       68
Closing Credits      56
COMMERCIAL BREAK     54
OPENING CREDITS      45
CLOSING CREDITS      38
(Pause.)             20
Name: count, dtype: int64

При этом перед отсутствующими значениями и после них чаще всего следуют короткие открывающие и закрывающие реплики персонажей, что может свидетельствовать о возможности их использования в качестве разделителей. Как минимум в том случае, когда отсутствующее значение соответствует рекламной паузе и разделяет 2 разных по контексту сюжета внутри эпизода.

In [6]:
df[df['Speaker'].isna()]['PrevText'].value_counts().head(10)

PrevText
 Okay.         66
END            45
 Yeah.         33
 Okay!         15
 All right.    14
 Hey!          14
 Yeah!         12
 Bye.          12
 What?         11
 Oh.           11
Name: count, dtype: int64

In [7]:
df[df['Speaker'].isna()]['NextText'].value_counts().head(10)

NextText
 Hey!                        53
 Hey.                        36
 What?                       29
 Hi!                         28
 Hi.                         27
End                          24
Commercial Break             20
 Andrew Reich & Ted Cohen    18
Ending Credits               18
COMMERCIAL BREAK             16
Name: count, dtype: int64

Таким образом я принял решение разменить отсутствующие значения специальной меткой, обозначающей разрыв в диалогах и, как следствие, разрыв в непрерывном контексте

In [8]:
BREAK_LABEL = 'Break'

df['Speaker'] = df['Speaker'].fillna(BREAK_LABEL)
df

,Text,Speaker,PrevText,NextText
0,Originally written by Marta Kauffman and David...,Break,None,Transcribed by guineapig.
1,Transcribed by guineapig.,Break,Originally written by Marta Kauffman and David...,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1,Transcribed by guineapig.,There's nothing to tell! He's just some guy I...
3,There's nothing to tell! He's just some guy I...,MONICA,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,"C'mon, you're going out with the guy! There's..."
4,"C'mon, you're going out with the guy! There's...",JOEY,There's nothing to tell! He's just some guy I...,So does he have a hump? A hump and a hairpiece?
...,...,...,...,...
69969,Then I'm happy too. (They're still hugging - ...,Ross,"Oh! Oh, I'm so happy.",COMMERCIAL BREAK
69970,COMMERCIAL BREAK,Break,Then I'm happy too. (They're still hugging - ...,Estelle's memorial service. Joey is giving a ...
69971,Estelle's memorial service. Joey is giving a ...,[Scene,COMMERCIAL BREAK,Thank you all for coming. We're here today to...
69972,Thank you all for coming. We're here today to...,Joey,Estelle's memorial service. Joey is giving a ...,THE END


Выводим статистику для авторов реплик. Видим, что часть одинаковых имен записаны разным способом, их нужно привести к одинаковой форме записи

In [9]:
df['Speaker'].value_counts().head(60)

Speaker
Rachel            7750
Ross              7437
Joey              6961
Chandler          6950
Monica            6876
Phoebe            6291
Break             6284
[Scene            2805
ROSS              1432
CHANDLER          1107
MONICA            1088
JOEY              1086
RACHEL            1024
PHOEBE             861
Mike               352
RACH               325
MNCA               259
CHAN               233
All                221
PHOE               195
Charlie            190
Transcribed by     170
Emily              167
Janice             158
Mr. Geller         147
Tag                146
RICHARD            128
Richard            126
Written by         124
David              120
Frank              119
Gunther            116
Amy                116
Paul               116
ALL                111
Mona               111
Mrs. Geller        111
CAROL              110
Pete               103
Joshua              98
Gary                96
Elizabeth           94
Woman               93
Jan

In [10]:
# Замена указанных в replace_cat_list значений признака на replace_name

def replace_with_cat_list(feature, replace_cat_list, replace_name='other'):
    mask = feature.isin(replace_cat_list)
    return feature.mask(mask, replace_name)

Поскольку пока неизвестно, какой персонаж будет выбран для обучения, проводим замену для всех персонажей, присутствующих в top60 по количеству реплик

In [11]:
names_for_replace = [
    (['RACHEL', 'RACH'], 'Rachel'),
    (['CHANDLER', 'CHAN'], 'Chandler'),
    (['ROSS'], 'Ross'),
    (['JOEY'], 'Joey'),
    (['MONICA', 'MNCA'], 'Monica'),
    (['PHOEBE', 'PHOE'], 'Phoebe'),
    (['ALL'], 'All'),
    (['RICHARD'], 'Richard'),
    (['MR. GELLER'], 'Mr. Geller'),
    (['MRS. GELLER'], 'Mrs. Geller'),
    (['JANICE'], 'Janice'),
    (['CAROL'], 'Carol'),
    (['GUNTHER'], 'Gunther'),
    (['SUSAN'], 'Susan'),
    (['JULIE'], 'Julie'),
    (['EDDIE'], 'Eddie'),
    (['SUSIE'], 'Susie'),
    (['BARRY'], 'Barry'),
    (['The Director'], 'Director'),
    (['[Scene', 'Transcribed by', 'Tag', 'Written by'], BREAK_LABEL),
    (['SCENE 1', 'SCENE 2', 'SCENE 3', 'SCENE 4'], BREAK_LABEL),
    (['SCENE 5', 'SCENE 6', 'SCENE 7', 'SCENE 8'], BREAK_LABEL)    
]

In [12]:
for names in names_for_replace:
    df.loc[:, ('Speaker')] = replace_with_cat_list(df.loc[:, ('Speaker')], names[0], names[1])

In [13]:
df['Speaker'].value_counts().head(60)

Speaker
Break          9563
Rachel         9099
Ross           8869
Chandler       8290
Monica         8223
Joey           8047
Phoebe         7347
Mike            352
All             332
Richard         254
Janice          208
Mr. Geller      204
Carol           192
Charlie         190
Emily           167
Mrs. Geller     167
Gunther         127
David           120
Frank           119
Amy             116
Paul            116
Mona            111
Pete            103
Susan           102
Joshua           98
Director         97
Gary             96
Elizabeth        94
Woman            93
Janine           92
Kathy            82
Joanna           73
Jill             69
Eric             68
Ben              68
Eddie            64
Gavin            64
Dr. Green        63
Kate             61
Danny            58
Mark             52
Ursula           51
Will             50
Doug             50
Man              49
Guy              49
Julie            49
Cecilia          49
Tim              46
Mr. Treeger 

Избавляемся от лишних BREAK_LABEL, идущих подряд. В результате каждый диалог со своим контекстом разделен только одной меткой разрыва контекста BREAK_LABEL. Удаляем ненужные столбцы, чтобы остались только реплики и их авторы.

In [14]:
df['NextSpeaker'] = df['Speaker'].shift(-1)
df

,Text,Speaker,PrevText,NextText,NextSpeaker
0,Originally written by Marta Kauffman and David...,Break,None,Transcribed by guineapig.,Break
1,Transcribed by guineapig.,Break,Originally written by Marta Kauffman and David...,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,Break
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,Break,Transcribed by guineapig.,There's nothing to tell! He's just some guy I...,Monica
3,There's nothing to tell! He's just some guy I...,Monica,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,"C'mon, you're going out with the guy! There's...",Joey
4,"C'mon, you're going out with the guy! There's...",Joey,There's nothing to tell! He's just some guy I...,So does he have a hump? A hump and a hairpiece?,Chandler
...,...,...,...,...,...
69969,Then I'm happy too. (They're still hugging - ...,Ross,"Oh! Oh, I'm so happy.",COMMERCIAL BREAK,Break
69970,COMMERCIAL BREAK,Break,Then I'm happy too. (They're still hugging - ...,Estelle's memorial service. Joey is giving a ...,Break
69971,Estelle's memorial service. Joey is giving a ...,Break,COMMERCIAL BREAK,Thank you all for coming. We're here today to...,Joey
69972,Thank you all for coming. We're here today to...,Joey,Estelle's memorial service. Joey is giving a ...,THE END,Break


In [15]:
fixed_df = df[~((df['Speaker']==BREAK_LABEL) & (df['NextSpeaker']==BREAK_LABEL))]
fixed_df = fixed_df.drop(['PrevText', 'NextText', 'NextSpeaker'], axis=1)
fixed_df

,Text,Speaker
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,Break
3,There's nothing to tell! He's just some guy I...,Monica
4,"C'mon, you're going out with the guy! There's...",Joey
5,So does he have a hump? A hump and a hairpiece?,Chandler
6,"Wait, does he eat chalk?",Phoebe
...,...,...
69968,"Oh! Oh, I'm so happy.",Rachel
69969,Then I'm happy too. (They're still hugging - ...,Ross
69971,Estelle's memorial service. Joey is giving a ...,Break
69972,Thank you all for coming. We're here today to...,Joey


Сохраняем получившийся датасет в csv чтобы дальше можно было работать с конкретным персонажем не повторяя анализ и очистку

In [16]:
df = fixed_df.to_csv('Friends_processed.csv', index=False)
df